In [13]:
import pandas as pd
import numpy as np
import json
import requests


def generate_path_API_1D(model, lake, start, stop, variables="T"):
    """
    Generate API path for 1D simulation data.
    
    Parameters:
    -----------
    model : str
        The simulation model (e.g., 'simstrat')
    lake : str
        The lake name (e.g., 'aegeri')
    start : str
        Start datetime in format 'YYYYMMDDHHMM' (e.g., '202405050300')
    stop : str
        Stop datetime in format 'YYYYMMDDHHMM' (e.g., '202406072300')
    variables : str, optional
        Variables to query (default: 'T' for temperature)
    
    Returns:
    --------
    str
        Complete API URL path
    """
    base_url = "https://alplakes-api.eawag.ch/simulations/1d/depthtime"
    return f"{base_url}/{model}/{lake}/{start}/{stop}?variables={variables}"


def read_API_1D_to_dataframe(api_url_or_json_path, variable='T'):
    """
    Read API JSON data and convert to pandas DataFrame with datetime index.
    
    This function reads JSON data from either:
    - A URL (API endpoint)
    - A local JSON file path
    
    The JSON structure should contain:
    - 'time': list of datetime strings
    - 'depth': dict with 'data' (list of depth values), 'unit', 'description'
    - 'variables': dict with variable names as keys (e.g., 'T' for temperature)
      Each variable contains 'data' (2D array), 'unit', 'description'
    
    Parameters:
    -----------
    api_url_or_json_path : str
        Either a URL to the API endpoint or a path to a local JSON file
    variable : str, optional
        Variable name to extract (default: 'T' for temperature)
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with:
        - First column: 'Datetime' (timezone-naive datetime64[ns])
        - Remaining columns: Depth values (e.g., '-81.000' to '-0.000')
        - Values: Temperature or other variable data
        
    Note: Datetime values are converted to timezone-naive for better compatibility
          with numpy and most pandas operations.
    
    Example:
    --------
    >>> # From API
    >>> url = generate_path_API_1D('simstrat', 'aegeri', '202405050300', '202406072300')
    >>> df = read_API_1D_to_dataframe(url)
    >>> print(df.columns)
    Index(['Datetime', '-81.000', '-80.000', ..., '-1.000', '-0.000'], dtype='object')
    
    >>> # From local file
    >>> df = read_API_1D_to_dataframe('ttt.json')
    """
    
    # Load JSON data
    if api_url_or_json_path.startswith('http'):
        # Fetch from API with timeout
        try:
            response = requests.get(api_url_or_json_path, timeout=300)  # 5 minute timeout
            response.raise_for_status()
            data = response.json()
        except requests.exceptions.Timeout:
            raise TimeoutError(
                "API request timed out. The date range might be too large. "
                "Try requesting smaller time periods (e.g., one year at a time)."
            )
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 504:
                raise TimeoutError(
                    f"API gateway timeout (504). The date range is too large. "
                    f"Try requesting smaller time periods (e.g., one year at a time) or use fetch_API_1D_chunked()."
                )
            else:
                raise
    else:
        # Load from local file
        with open(api_url_or_json_path, 'r') as f:
            data = json.load(f)
    
    # Extract time, depth, and variable data
    time = data['time']
    depth_values = data['depth']['data']
    variable_data = data['variables'][variable]['data']
    
    # Convert depth values to negative (representing depth below surface)
    # Format with 3 decimal places
    depth_values_negative = [f"{-d:.3f}" for d in depth_values]
    
    # The API returns data as [depth x time], we need [time x depth]
    # So we need to transpose the data
    variable_data_transposed = np.array(variable_data).T
    
    # Create DataFrame
    # Rows = time points, Columns = depth values (negative, 3 decimals)
    df = pd.DataFrame(variable_data_transposed, columns=depth_values_negative)
    
    # Convert time strings to datetime and remove timezone (convert to naive datetime)
    datetime_series = pd.to_datetime(time).tz_localize(None)
    
    # Add Datetime as a column (not index)
    df.insert(0, 'Datetime', datetime_series)
    
    return df


def fetch_API_1D_chunked(model, lake, start, stop, variables="T", chunk_months=12, save_csv=None):
    """
    Fetch large date ranges from API in smaller chunks and combine them.
    
    This function automatically splits large date ranges into smaller chunks
    to avoid API timeouts.
    
    Parameters:
    -----------
    model : str
        The simulation model (e.g., 'simstrat')
    lake : str
        The lake name (e.g., 'geneva')
    start : str
        Start datetime in format 'YYYYMMDDHHMM' (e.g., '198101010000')
    stop : str
        Stop datetime in format 'YYYYMMDDHHMM' (e.g., '202406072300')
    variables : str, optional
        Variables to query (default: 'T' for temperature)
    chunk_months : int, optional
        Number of months per chunk (default: 12 for yearly chunks)
    save_csv : str, optional
        Path to save the resulting DataFrame as CSV (default: None, no save)
        Example: 'output.csv' or '/path/to/file.csv'
    
    Returns:
    --------
    pd.DataFrame
        Combined DataFrame with all data
    
    Example:
    --------
    >>> # Fetch 43 years of data in yearly chunks
    >>> df = fetch_API_1D_chunked('simstrat', 'geneva', '198101010000', '202406072300')
    
    >>> # Fetch and save to CSV
    >>> df = fetch_API_1D_chunked('simstrat', 'geneva', '198101010000', '202406072300', 
    ...                            save_csv='geneva_1981_2024.csv')
    """
    from datetime import datetime, timedelta
    from dateutil.relativedelta import relativedelta
    
    # Parse start and stop dates
    start_dt = datetime.strptime(start, '%Y%m%d%H%M')
    stop_dt = datetime.strptime(stop, '%Y%m%d%H%M')
    
    dfs = []
    current_start = start_dt
    
    print(f"Fetching data from {start_dt} to {stop_dt} in {chunk_months}-month chunks...")
    
    chunk_count = 0
    while current_start < stop_dt:
        # Calculate chunk end (either chunk_months ahead or final stop date)
        current_stop = min(
            current_start + relativedelta(months=chunk_months),
            stop_dt
        )
        
        # Format dates back to API format
        chunk_start_str = current_start.strftime('%Y%m%d%H%M')
        chunk_stop_str = current_stop.strftime('%Y%m%d%H%M')
        
        # Generate URL and fetch
        url = generate_path_API_1D(model, lake, chunk_start_str, chunk_stop_str, variables)
        
        chunk_count += 1
        print(f"  Chunk {chunk_count}: {current_start.date()} to {current_stop.date()}...", end=' ')
        
        try:
            df_chunk = read_API_1D_to_dataframe(url, variable=variables)
            dfs.append(df_chunk)
            print(f"✓ ({len(df_chunk)} rows)")
        except Exception as e:
            print(f"✗ Error: {e}")
            # Continue with next chunk even if this one fails
        
        # Move to next chunk
        current_start = current_stop
    
    if not dfs:
        raise ValueError("No data was successfully fetched")
    
    # Combine all chunks
    print(f"\nCombining {len(dfs)} chunks...")
    df_combined = pd.concat(dfs, axis=0, ignore_index=True)
    
    # Remove any duplicate timestamps (at chunk boundaries)
    df_combined = df_combined.drop_duplicates(subset=['Datetime'], keep='first')
    
    # Reset index after dropping duplicates
    df_combined = df_combined.reset_index(drop=True)
    
    print(f"✓ Total: {len(df_combined)} rows")
    
    # Save to CSV if requested
    if save_csv:
        print(f"\nSaving to CSV: {save_csv}...")
        # Create directory if it doesn't exist
        import os
        csv_dir = os.path.dirname(save_csv)
        if csv_dir and not os.path.exists(csv_dir):
            os.makedirs(csv_dir, exist_ok=True)
            print(f"  Created directory: {csv_dir}")
        df_combined.to_csv(save_csv, index=False)
        print(f"✓ Saved successfully")
    
    return df_combined


def load_lake_data(model, lake, start, stop, variables="T", 
                   local_path=None, save_csv=None, chunk_months=12, 
                   force_download=False):
    """
    Smart function to load lake simulation data from local file or API.
    
    This function automatically decides whether to:
    1. Load from a local file (if it exists and is not stale)
    2. Download from the API (if local file doesn't exist or is outdated)
    
    Parameters:
    -----------
    model : str
        The simulation model (e.g., 'simstrat')
    lake : str
        The lake name (e.g., 'geneva', 'aegeri')
    start : str
        Start datetime in format 'YYYYMMDDHHMM'
    stop : str
        Stop datetime in format 'YYYYMMDDHHMM'
    variables : str, optional
        Variables to query (default: 'T' for temperature)
    local_path : str, optional
        Path to check for local data file. Can be:
        - CSV file: 'data/geneva/temperature.csv'
        - DAT file: 'data/geneva/T_out.dat'
        - Directory: 'data/geneva' (will look for T_out.dat or *.csv)
        If None, always downloads from API
    save_csv : str, optional
        Path to save downloaded data as CSV (default: None)
    chunk_months : int, optional
        Number of months per chunk when downloading (default: 12)
    force_download : bool, optional
        If True, always download from API even if local file exists (default: False)
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with lake simulation data
    
    Examples:
    ---------
    >>> # Try local first, download if needed
    >>> df = load_lake_data('simstrat', 'geneva', '198101010000', '202406072300',
    ...                      local_path='data/Geneva/T_out.dat')
    
    >>> # Always download and save
    >>> df = load_lake_data('simstrat', 'geneva', '198101010000', '202406072300',
    ...                      force_download=True, save_csv='geneva_new.csv')
    
    >>> # Download if no local file exists
    >>> df = load_lake_data('simstrat', 'aegeri', '202001010000', '202406072300',
    ...                      local_path='data/aegeri/', save_csv='data/aegeri/temperature.csv')
    """
    import os
    
    # Determine if we should try to load from local file
    should_load_local = False
    local_file_path = None
    
    if local_path and not force_download:
        if os.path.isdir(local_path):
            # Check for common filenames in the directory
            possible_files = [
                os.path.join(local_path, 'T_out.dat'),
                os.path.join(local_path, 'temperature.csv'),
                os.path.join(local_path, f'{lake}_temperature.csv'),
            ]
            for file_path in possible_files:
                if os.path.exists(file_path):
                    local_file_path = file_path
                    should_load_local = True
                    break
        elif os.path.isfile(local_path):
            local_file_path = local_path
            should_load_local = True
    
    # Load from local file if available
    if should_load_local:
        print(f"Loading data from local file: {local_file_path}")
        file_ext = os.path.splitext(local_file_path)[1].lower()
        
        if file_ext == '.csv':
            # Load CSV file
            df = pd.read_csv(local_file_path)
            if 'Datetime' in df.columns:
                df['Datetime'] = pd.to_datetime(df['Datetime'])
            print(f"✓ Loaded {len(df)} rows from CSV")
            
        elif file_ext == '.dat':
            # Load DAT file (Simstrat format)
            # Format: First column is days since reference date
            df = pd.read_csv(local_file_path)
            
            # Get the time column (first column)
            time_col = df.columns[0]
            time_values = df[time_col]
            
            # Get depth columns (all columns except first)
            depth_columns = df.columns[1:].tolist()
            
            # Convert first column (days) to datetime
            # Assuming reference date is 1981-01-01 (common for Simstrat)
            datetime_values = pd.Timestamp('1981-01-01') + pd.to_timedelta(time_values, unit='D')
            
            # Create new column names with proper formatting
            new_columns = ['Datetime']
            for col in depth_columns:
                try:
                    depth = float(col)
                    # Assume positive values in DAT file should be negative (depth below surface)
                    if depth >= 0:
                        new_columns.append(f'-{depth:.3f}')
                    else:
                        new_columns.append(f'{depth:.3f}')
                except:
                    # Keep non-numeric column names as-is
                    new_columns.append(col)
            
            # Create new dataframe with proper structure
            data_dict = {'Datetime': datetime_values}
            for i, old_col in enumerate(depth_columns):
                data_dict[new_columns[i+1]] = df[old_col].values
            
            df = pd.DataFrame(data_dict)
            
            print(f"✓ Loaded {len(df)} rows from DAT file")
        
        else:
            raise ValueError(f"Unsupported file format: {file_ext}. Use .csv or .dat")
        
        # Filter by date range
        start_dt = pd.to_datetime(start, format='%Y%m%d%H%M')
        stop_dt = pd.to_datetime(stop, format='%Y%m%d%H%M')
        
        df = df[(df['Datetime'] >= start_dt) & (df['Datetime'] <= stop_dt)]
        print(f"✓ Filtered to date range: {len(df)} rows between {start_dt.date()} and {stop_dt.date()}")
        
        return df
    
    # Download from API
    else:
        if force_download:
            print(f"Force download enabled - fetching from API...")
        elif local_path:
            print(f"Local file not found at: {local_path}")
            print(f"Downloading from API...")
        else:
            print(f"No local path specified - downloading from API...")
        
        # Create save directory if needed
        if save_csv:
            csv_dir = os.path.dirname(save_csv)
            if csv_dir and not os.path.exists(csv_dir):
                os.makedirs(csv_dir, exist_ok=True)
                print(f"✓ Created directory: {csv_dir}")
        
        df = fetch_API_1D_chunked(
            model=model,
            lake=lake,
            start=start,
            stop=stop,
            variables=variables,
            chunk_months=chunk_months,
            save_csv=save_csv
        )
        
        return df



In [14]:
df = load_lake_data(
    model='simstrat',
    lake='aegeri',
    start='201901010000',      # ✓ String, 12 digits
    stop='202101010000',       # ✓ String, 12 digits  
    variables="T",
    local_path='data/Aegeri',  # Will check here first
    save_csv='data/Aegeri/aegeri.csv',  # Directory created automatically! ✓
    chunk_months=12,
    force_download=False
)

Local file not found at: data/Aegeri
✓ Created directory: data/Aegeri
Fetching data from 2019-01-01 00:00:00 to 2021-01-01 00:00:00 in 12-month chunks...
  Chunk 1: 2019-01-01 to 2020-01-01... ✓ (2920 rows)
  Chunk 2: 2020-01-01 to 2021-01-01... ✓ (2928 rows)

Combining 2 chunks...
✓ Total: 5848 rows

Saving to CSV: data/Aegeri/aegeri.csv...
✓ Saved successfully


In [8]:
df

,Datetime,-81.000,-80.000,-79.000,-78.000,-77.000,-76.000,-75.000,-74.000,-73.000,...,-9.000,-8.000,-7.000,-6.000,-5.000,-4.000,-3.000,-2.000,-1.000,-0.000
0,2019-01-01 01:00:02.880,5.952,5.952,5.952,5.952,5.952,5.951,5.951,5.950,5.950,...,5.897,5.896,5.894,5.893,5.891,5.890,5.888,5.885,5.882,5.876
1,2019-01-01 04:00:02.880,5.949,5.949,5.949,5.949,5.949,5.948,5.948,5.948,5.947,...,5.890,5.888,5.886,5.885,5.883,5.881,5.879,5.876,5.872,5.866
2,2019-01-01 07:00:02.880,5.947,5.946,5.946,5.946,5.946,5.945,5.945,5.945,5.944,...,5.883,5.882,5.880,5.878,5.876,5.875,5.872,5.870,5.866,5.860
3,2019-01-01 10:00:02.880,5.944,5.944,5.944,5.943,5.943,5.943,5.942,5.942,5.942,...,5.883,5.882,5.881,5.880,5.879,5.878,5.877,5.876,5.874,5.870
4,2019-01-01 13:00:02.880,5.942,5.942,5.942,5.941,5.941,5.941,5.940,5.940,5.940,...,5.893,5.894,5.894,5.895,5.896,5.897,5.897,5.897,5.896,5.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5843,2020-12-31 10:00:02.880,6.021,6.021,6.021,6.020,6.020,6.020,6.019,6.018,6.018,...,5.925,5.923,5.922,5.921,5.919,5.918,5.916,5.914,5.911,5.906
5844,2020-12-31 13:00:02.880,6.016,6.016,6.016,6.016,6.015,6.015,6.014,6.014,6.013,...,5.936,5.936,5.935,5.935,5.934,5.934,5.933,5.932,5.931,5.927
5845,2020-12-31 16:00:02.880,6.012,6.011,6.011,6.011,6.010,6.010,6.009,6.009,6.008,...,5.937,5.935,5.934,5.933,5.931,5.929,5.927,5.924,5.920,5.912
5846,2020-12-31 19:00:02.880,6.006,6.006,6.006,6.006,6.005,6.004,6.004,6.004,6.003,...,5.925,5.923,5.921,5.919,5.916,5.914,5.912,5.908,5.904,5.896
